In [66]:
import numpy as np
import pandas as pd
import scipy
import seaborn as sns

sns.set_theme(style='white')

In [67]:
data = pd.read_excel('data/BakeryData_Vilnius.xlsx')
data['date'] = pd.to_datetime(data['date'], format='%Y-%m-%d')
data

,date,weekday,main street A,main street B,station A,station B
0,2016-05-11,3,2.23,NaN,NaN,NaN
1,2016-05-12,4,18.10,NaN,NaN,NaN
2,2016-05-13,5,15.85,NaN,NaN,NaN
3,2016-05-14,6,14.22,NaN,NaN,NaN
4,2016-05-15,7,2.58,NaN,NaN,NaN
...,...,...,...,...,...,...
2572,2023-05-27,6,168.05,32.34,76.97,114.30
2573,2023-05-28,7,44.62,32.85,80.21,91.25
2574,2023-05-29,1,64.11,116.84,149.75,92.56
2575,2023-05-30,2,103.63,134.48,194.03,75.63


In [68]:
# filter out corona data and data before 2017
def filter_out_data(data):
    start_corona_remove = pd.to_datetime('2020-10-01')
    end_corona_remove = pd.to_datetime('2022-10-01')
    start_remove = pd.to_datetime('2017-01-01')
    filtered_data = data.loc[(data.date < start_corona_remove) | (data.date > end_corona_remove)]
    filtered_data = filtered_data.loc[(filtered_data.date > start_remove)]
    return filtered_data


def calculate_tau(p, c, cs, pl):
    p_tilde = p + cs + pl
    c_tilde = c + cs + pl
    return (p_tilde - c_tilde) / p_tilde


def bootstrap_interval_estimate_normal(tau, params):
    estimations = []
    for _ in range(M):
        data = scipy.stats.norm.rvs(size=n, loc=params[0], scale=params[1])
        generated_params = scipy.stats.norm.fit(data)
        estimate = scipy.stats.norm.ppf(tau, loc=generated_params[0], scale=generated_params[1])
        estimations.append(estimate)

    [lower, upper] = np.quantile(estimations, [alpha / 2, 1 - alpha / 2])
    return lower, upper


def bootstrap_interval_estimate_lognormal(tau, params):
    estimations = []
    for _ in range(M):
        data = scipy.stats.lognorm.rvs(size=n, s=params[0], loc=params[1], scale=params[2])
        generated_params = scipy.stats.lognorm.fit(data)
        estimate = scipy.stats.lognorm.ppf(tau, s=generated_params[0], loc=generated_params[1],
                                           scale=generated_params[2])
        estimations.append(estimate)

    [lower, upper] = np.quantile(estimations, [alpha / 2, 1 - alpha / 2])
    return lower, upper


def bootstrap_interval_estimate_gamma(tau, params):
    estimations = []
    for _ in range(M):
        data = scipy.stats.gamma.rvs(size=n, a=params[0], loc=params[1], scale=params[2])
        generated_params = scipy.stats.gamma.fit(data)
        estimate = scipy.stats.gamma.ppf(tau, a=generated_params[0], loc=generated_params[1], scale=generated_params[2])
        estimations.append(estimate)

    [lower, upper] = np.quantile(estimations, [alpha / 2, 1 - alpha / 2])
    return lower, upper


def quantity_estimate_nonparametric(data, tau):
    position = int(np.ceil(n * tau))
    return sorted(data)[position]


def interval_estimate_nonparametric(data, alpha, tau):
    quantile = scipy.stats.norm.ppf(1 - alpha / 2)
    diapason = quantile * np.sqrt(len(data) * tau * (1 - tau))
    lower = int(np.ceil(n * tau - diapason))
    upper = int(np.ceil(n * tau + diapason))
    return sorted(data)[lower], sorted(data)[upper]

In [69]:
streetAData = data[['date', 'weekday', 'main street A']].rename(columns={'main street A': 'values'}).dropna()
stationAData = data[['date', 'weekday', 'station A']].rename(columns={'station A': 'values'}).dropna()

filtered_data_streeta = filter_out_data(streetAData)
filtered_data_stationa = filter_out_data(stationAData)

In [70]:
# Main Store A
other_days, saturdays = [x for _, x in filtered_data_streeta.groupby(filtered_data_streeta['weekday'] == 6)]
other_days, fridays = [x for _, x in other_days.groupby(filtered_data_streeta['weekday'] == 5)]

# Station A
weekend, weekdays = [x for _, x in filtered_data_stationa.groupby(filtered_data_streeta['weekday'] <= 5)]

In [71]:
fridays_params = scipy.stats.norm.fit(fridays['values'])
saturdays_params = scipy.stats.norm.fit(saturdays['values'])
other_days_params = scipy.stats.lognorm.fit(other_days['values'])

tau_street_a = calculate_tau(4.64, 3.85, 0.11, 0.15)

fridays_optimal = scipy.stats.norm.ppf(tau_street_a, loc=fridays_params[0], scale=fridays_params[1])
saturdays_optimal = scipy.stats.norm.ppf(tau_street_a, loc=saturdays_params[0], scale=saturdays_params[1])
other_days_optimal = scipy.stats.lognorm.ppf(tau_street_a, s=other_days_params[0], loc=other_days_params[1],
                                             scale=other_days_params[2])

print("Tau Main street A", tau_street_a)
print("Optimal quantity for Fridays", fridays_optimal)
print("Optimal quantity for Saturdays", saturdays_optimal)
print("Optimal quantity for other days", other_days_optimal)

Tau Main street A 0.16122448979591836
Optimal quantity for Fridays 93.8357081278314
Optimal quantity for Saturdays 138.23533164833452
Optimal quantity for other days 41.26652937704263


In [72]:
weekdays_params = scipy.stats.gamma.fit(weekdays['values'])
weekend_params = scipy.stats.norm.fit(weekend['values'])

tau_station_a = calculate_tau(4.16, 3.32, 0.09, 0.15)

weekdays_optimal = scipy.stats.gamma.ppf(tau_station_a, a=weekdays_params[0], loc=weekdays_params[1],
                                         scale=weekdays_params[2])
weekend_optimal = scipy.stats.norm.ppf(tau_station_a, loc=weekend_params[0], scale=weekend_params[1])

print("Tau Station A", tau_station_a)
print("Optimal quantity for weekdays", weekdays_optimal)
print("Optimal quantity for weekend", weekend_optimal)

Tau Station A 0.19090909090909106
Optimal quantity for weekdays 125.83470187445616
Optimal quantity for weekend 67.32772261478024


In [73]:
# Parametric bootstrap interval estimation
M = 10000
n = 1000
alpha = 0.05

fridays_interval = bootstrap_interval_estimate_normal(tau_street_a, fridays_params)
saturdays_interval = bootstrap_interval_estimate_normal(tau_street_a, saturdays_params)
other_days_interval = bootstrap_interval_estimate_lognormal(tau_street_a, other_days_params)

In [74]:
print("Main Street A - Fridays - inteval", fridays_interval)
print("Main Street A - Saturdays - inteval", saturdays_interval)
print("Main Street A - other days - inteval", other_days_interval)

Main Street A - Fridays - inteval (93.46534999251541, 94.21616613387187)
Main Street A - Saturdays - inteval (137.24131129687237, 139.25323698065029)
Main Street A - other days - inteval (40.24757763316725, 42.21806854225656)


In [75]:
weekdays_interval = bootstrap_interval_estimate_gamma(tau_station_a, weekdays_params)
weekend_interval = bootstrap_interval_estimate_normal(tau_station_a, weekend_params)
print("Station A - weekdays - inteval", weekdays_interval)
print("Station A - weekend - inteval", weekend_interval)

Station A - weekdays - inteval (123.88660316915063, 127.89243957264465)
Station A - weekend - inteval (66.703004655506, 67.94848547463582)


In [76]:
# non-parametric estimates

fridays_optimal_nonparam = quantity_estimate_nonparametric(fridays['values'], tau_street_a)
fridays_interval_nonparam = interval_estimate_nonparametric(fridays['values'], 0.05, tau_street_a)

saturdays_optimal_nonparam = quantity_estimate_nonparametric(saturdays['values'], tau_street_a)
saturdays_interval_nonparam = interval_estimate_nonparametric(saturdays['values'], 0.05, tau_street_a)

other_days_optimal_nonparam = quantity_estimate_nonparametric(other_days['values'], tau_street_a)
other_days_interval_nonparam = interval_estimate_nonparametric(other_days['values'], 0.05, tau_street_a)

weekdays_optimal_nonparam = quantity_estimate_nonparametric(weekdays['values'], tau_station_a)
weekdays_interval_nonparam = interval_estimate_nonparametric(weekdays['values'], 0.05, tau_station_a)

weekend_optimal_nonparam = quantity_estimate_nonparametric(weekend['values'], tau_station_a)
weekend_interval_nonparam = interval_estimate_nonparametric(weekend['values'], 0.05, tau_station_a)

In [77]:
print("Fridays non-parametric estimate", fridays_optimal_nonparam, fridays_interval_nonparam)
print("Saturdays non-parametric estimate", saturdays_optimal_nonparam, saturdays_interval_nonparam)
print("Other days non-parametric estimate", other_days_optimal_nonparam, other_days_interval_nonparam)
print("Weekdays non-parametric estimate", weekdays_optimal_nonparam, weekdays_interval_nonparam)
print("Weekend non-parametric estimate", weekend_optimal_nonparam, weekend_interval_nonparam)

Fridays non-parametric estimate 101.34 (100.58, 102.18)
Saturdays non-parametric estimate 158.44 (156.47, 160.28)
Other days non-parametric estimate 40.75 (39.6, 41.67)
Weekdays non-parametric estimate 132.3 (130.4, 135.49)
Weekend non-parametric estimate 78.3 (76.97, 79.84)
